<a href="https://colab.research.google.com/github/pratim808/smol-course/blob/main/1_instruction_tuning/notebooks/sft_finetuning_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Supervised Fine-Tuning with SFTTrainer

This notebook demonstrates how to fine-tune the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer` from the `trl` library. The notebook cells run and will finetune the model. You can select your difficulty by trying out different datasets.

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Exercise: Fine-Tuning SmolLM2 with SFTTrainer</h2>
    <p>Take a dataset from the Hugging Face hub and finetune a model on it. </p>
    <p><b>Difficulty Levels</b></p>
    <p>🐢 Use the `HuggingFaceTB/smoltalk` dataset</p>
    <p>🐕 Try out the `bigcode/the-stack-smol` dataset and finetune a code generation model on a specific subset `data/python`.</p>
    <p>🦁 Select a dataset that relates to a real world use case your interested in</p>
</div>

In [4]:
# Install the requirements in Google Colab
#!pip install transformers datasets trl huggingface_hub

# Authenticate to Hugging Face

from huggingface_hub import login
login()

# for convenience you can create an environment variable containing your hub token as HF_TOKEN

In [2]:
!pip install -U transformers datasets trl huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.1
    Uninstalling transformers-4.47.1:
      Successfully uninstalled transformers-4.47.1
ERROR: pip's d

In [2]:
# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Load the model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Set up the chat format
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2-FT-MyDataset"
finetune_tags = ["smol-course", "module_1"]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/704 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.66k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/831 [00:00<?, ?B/s]

# Generate with the base model

Here we will try out the base model which does not have a chat template.

In [3]:
# Let's test the base model before training
prompt = "Write a haiku about programming"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)
print("Before training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Before training:
user
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a


In [7]:
tokenizer.decode(outputs[0], skip_special_tokens=True)

'user\nWrite a haiku about programming\nWrite a haiku about programming\nWrite a haiku about programming\nWrite a haiku about programming\nWrite a haiku about programming\nWrite a haiku about programming\nWrite a haiku about programming\nWrite a haiku about programming\nWrite a haiku about programming\nWrite a haiku about programming\nWrite a haiku about programming\nWrite a haiku about programming\nWrite a haiku about programming\nWrite a haiku about programming\nWrite a haiku about programming\nWrite a'

## Dataset Preparation

We will load a sample dataset and format it for training. The dataset should be structured with input-output pairs, where each input is a prompt and the output is the expected response from the model.

**TRL will format input messages based on the model's chat templates.** They need to be represented as a list of dictionaries with the keys: `role` and `content`,.

In [8]:
# Load a sample dataset
from datasets import load_dataset

# TODO: define your dataset and config using the path and name parameters
ds = load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations")

README.md:   0%|          | 0.00/9.25k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/946k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/52.6k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2260 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/119 [00:00<?, ? examples/s]

In [9]:
ds

DatasetDict({
    train: Dataset({
        features: ['full_topic', 'messages'],
        num_rows: 2260
    })
    test: Dataset({
        features: ['full_topic', 'messages'],
        num_rows: 119
    })
})

In [ ]:
# TODO: 🦁 If your dataset is not in a format that TRL can convert to the chat template, you will need to process it. Refer to the [module](../chat_templates.md)

In [10]:
def formatting_prompts_func(examples):
    convos = examples["messages"]
    texts = [
        tokenizer.apply_chat_template(
            convo, tokenize = False, add_generation_prompt = False
        )
        for convo in convos
    ]
    return { "text" : texts, }
pass
ds = ds.map(
    formatting_prompts_func,
    batched=True,
)

Map:   0%|          | 0/2260 [00:00<?, ? examples/s]

Map:   0%|          | 0/119 [00:00<?, ? examples/s]

In [11]:
ds

DatasetDict({
    train: Dataset({
        features: ['full_topic', 'messages', 'text'],
        num_rows: 2260
    })
    test: Dataset({
        features: ['full_topic', 'messages', 'text'],
        num_rows: 119
    })
})

## Configuring the SFTTrainer

The `SFTTrainer` is configured with various parameters that control the training process. These include the number of training steps, batch size, learning rate, and evaluation strategy. Adjust these parameters based on your specific requirements and computational resources.

In [14]:
# Configure the SFTTrainer
sft_config = SFTConfig(
    output_dir="./sft_output",

    max_steps=1,  # Adjust based on dataset size and desired training duration
    per_device_train_batch_size=4,  # Set according to your GPU memory capacity
    learning_rate=5e-5,  # Common starting point for fine-tuning
    logging_steps=10,  # Frequency of logging training metrics
    save_steps=100,# Frequency of saving model checkpoints
    report_to = "none", # Use this for WandB etc
    evaluation_strategy="steps",  # Evaluate the model at regular intervals
    eval_steps=50,  # Frequency of evaluation
    use_mps_device=(
        True if device == "mps" else False
    ),  # Use MPS for mixed precision training
    hub_model_id=finetune_name,  # Set a unique name for your model
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    #dataset_text_field = "text",
    args=sft_config,
    train_dataset=ds["train"],
    tokenizer=tokenizer,
    eval_dataset=ds["test"],
)

# TODO: 🦁 🐕 align the SFTTrainer params with your chosen dataset. For example, if you are using the `bigcode/the-stack-smol` dataset, you will need to choose the `content` column`

<ipython-input-14-dee31c45aaf8>:20: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Map:   0%|          | 0/2260 [00:00<?, ? examples/s]

## Training the Model

With the trainer configured, we can now proceed to train the model. The training process will involve iterating over the dataset, computing the loss, and updating the model's parameters to minimize this loss.

In [15]:
# Train the model
trainer.train()

# Save the model
trainer.save_model(f"./{finetune_name}")

Step,Training Loss,Validation Loss


In [ ]:
trainer.push_to_hub(tags=finetune_tags)

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

checkpoint = "NousResearch/Hermes-2-Pro-Llama-3-8B"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint, torch_dtype=torch.bfloat16, device_map="auto")

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Bonus Exercise: Generate with fine-tuned model</h2>
    <p>🐕 Use the fine-tuned to model generate a response, just like with the base example..</p>
</div>

In [16]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

checkpoint = "/content/sft_output/checkpoint-1"

tokenizer_ft = AutoTokenizer.from_pretrained(checkpoint)
model_ft = AutoModelForCausalLM.from_pretrained(checkpoint, torch_dtype=torch.bfloat16, device_map="auto")

In [17]:
# Test the fine-tuned model on the same prompt

# Let's test the base model before training
prompt = "Write a haiku about programming"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer_ft.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer_ft(formatted_prompt, return_tensors="pt").to(device)

outputs = model_ft.generate(**inputs, max_new_tokens=100)
print("Before training:")
print(tokenizer_ft.decode(outputs[0],skip_special_tokens=True))


# TODO: use the fine-tuned to model generate a response, just like with the base example.

Before training:
user
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a


## 💐 You're done!

This notebook provided a step-by-step guide to fine-tuning the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer`. By following these steps, you can adapt the model to perform specific tasks more effectively. If you want to carry on working on this course, here are steps you could try out:

- Try this notebook on a harder difficulty
- Review a colleagues PR
- Improve the course material via an Issue or PR.

#Sft Finetuning using `bigcode/the-stack-smol` dataset

In [1]:
# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Load the model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Set up the chat format
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2-FT-the-stack-smol"
finetune_tags = ["smol-course", "module_1"]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/704 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.66k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/831 [00:00<?, ?B/s]

In [19]:
# Let's test the base model before training
prompt = "how are you"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)
print("Before training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Before training:
user
how are you
how are you
how are you
how are you
how are you
how are you
how are you
how are you
how are you
how are you
how are you
how are you
how are you
how are you
how are you
how are you
how are you
how are you
how are you
how are you
how are you
how are you
how are you
how are you
how are you
how are you



In [5]:
from datasets import load_dataset
ds = load_dataset("bigcode/the-stack-smol", data_dir="data/python", split="train")

data.json:   0%|          | 0.00/87.0M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [6]:
ds

Dataset({
    features: ['content', 'avg_line_length', 'max_line_length', 'alphanum_fraction', 'licenses', 'repository_name', 'path', 'size', 'lang'],
    num_rows: 10000
})

In [7]:
ds[0]['content']

'# Copyright 2020 gRPC authors.\n#\n# Licensed under the Apache License, Version 2.0 (the "License");\n# you may not use this file except in compliance with the License.\n# You may obtain a copy of the License at\n#\n#     http://www.apache.org/licenses/LICENSE-2.0\n#\n# Unless required by applicable law or agreed to in writing, software\n# distributed under the License is distributed on an "AS IS" BASIS,\n# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.\n# See the License for the specific language governing permissions and\n# limitations under the License.\n"""The Python AsyncIO implementation of the GRPC helloworld.Greeter server."""\n\nimport logging\nimport asyncio\nimport grpc\n\nimport helloworld_pb2\nimport helloworld_pb2_grpc\n\n\nclass Greeter(helloworld_pb2_grpc.GreeterServicer):\n\n    async def SayHello(\n            self, request: helloworld_pb2.HelloRequest,\n            context: grpc.aio.ServicerContext) -> helloworld_pb2.HelloReply:\n        re

In [8]:
# Select the first 50 samples from the 'train' split
subset = ds.select(range(50))
subset

Dataset({
    features: ['content', 'avg_line_length', 'max_line_length', 'alphanum_fraction', 'licenses', 'repository_name', 'path', 'size', 'lang'],
    num_rows: 50
})

In [9]:
def process_dataset(sample):
    # 1. Create a message format with the role and content
    messages = [
        {"role": "user", "content": sample["content"]},
        #{"role": "assistant", "content": sample["answer"]},
    ]

    # 2. Apply the chat template to the samples using the tokenizer's method
    formatted_text = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=False
    )
    return {"text": formatted_text}


#ds_1 = load_dataset("openai/gsm8k", "main", split="test")
subset = subset.map(process_dataset)
subset

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Dataset({
    features: ['content', 'avg_line_length', 'max_line_length', 'alphanum_fraction', 'licenses', 'repository_name', 'path', 'size', 'lang', 'text'],
    num_rows: 50
})

In [11]:
subset = subset.select_columns(["content", "text"])
subset

Dataset({
    features: ['content', 'text'],
    num_rows: 50
})

In [55]:
subset['text'][0]

"<|im_start|>user\n.global s_prepare_buffers\ns_prepare_buffers:\npush %r10\npush %r11\npush %r13\npush %r15\npush %r9\npush %rbx\npush %rcx\npush %rdi\npush %rsi\nlea addresses_D_ht+0x1532, %rbx\nnop\nsub %r15, %r15\nmovw $0x6162, (%rbx)\nnop\nnop\nnop\nnop\nnop\nadd %r13, %r13\nlea addresses_UC_ht+0xac72, %rcx\nnop\nnop\nnop\nnop\nnop\nxor $63492, %r10\nmovw $0x6162, (%rcx)\nnop\nnop\ncmp $52783, %r15\nlea addresses_A_ht+0xd7f2, %r9\nclflush (%r9)\nnop\nnop\nnop\nnop\nnop\nxor %rcx, %rcx\nmov (%r9), %r11d\nnop\nnop\nnop\nnop\nnop\ncmp %rcx, %rcx\nlea addresses_normal_ht+0x1a2aa, %r13\ncmp $31476, %rcx\nmovb $0x61, (%r13)\nnop\nnop\nadd %r10, %r10\nlea addresses_UC_ht+0x672, %rsi\nlea addresses_UC_ht+0x1e272, %rdi\nclflush (%rdi)\nnop\nnop\nnop\nnop\ninc %r11\nmov $39, %rcx\nrep movsw\nnop\nxor %r11, %r11\nlea addresses_A_ht+0x2372, %rsi\nlea addresses_D_ht+0x1ed88, %rdi\ncmp $43777, %rbx\nmov $68, %rcx\nrep movsb\nnop\nadd %r13, %r13\nlea addresses_WT_ht+0x8172, %rcx\nnop\nnop\nnop\n

#Configuring the SFTTrainer
The SFTTrainer is configured with various parameters that control the training process. These include the number of training steps, batch size, learning rate, and evaluation strategy. Adjust these parameters based on your specific requirements and computational resources.

In [12]:
from trl import SFTTrainer,SFTConfig
from transformers import TrainingArguments
# Configure the SFTTrainer

from trl import SFTTrainer
from transformers import TrainingArguments
# Configure the SFTTrainer

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=SFTConfig(
        output_dir="./sft_output_1",
        max_steps=2,  # Adjust based on dataset size and desired training duration
        per_device_train_batch_size=4,  # Set according to your GPU memory capacity
        learning_rate=5e-5,  # Common starting point for fine-tuning
        logging_steps=10,  # Frequency of logging training metrics
        save_steps=100,# Frequency of saving model checkpoints
        report_to = "none", # Use this for WandB etc
        #evaluation_strategy="steps",  # Evaluate the model at regular intervals
        #eval_steps=50,  # Frequency of evaluation
        use_mps_device=(
            True if device == "mps" else False
        ),  # Use MPS for mixed precision training
        hub_model_id=finetune_name,  # Set a unique name for your model
    ),
    train_dataset=subset,
    #dataset_text_field = "text",
    tokenizer=tokenizer,
    #eval_dataset=ds["test"],
)


<ipython-input-12-2f27f0eb3eea>:10: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Map:   0%|          | 0/50 [00:00<?, ? examples/s]

## Training the Model

With the trainer configured, we can now proceed to train the model. The training process will involve iterating over the dataset, computing the loss, and updating the model's parameters to minimize this loss.

In [13]:
# Train the model
trainer.train()

# Save the model
trainer.save_model(f"./{finetune_name}")

Step,Training Loss


In [14]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

checkpoint = "/content/sft_output_1/checkpoint-2"

tokenizer_ft = AutoTokenizer.from_pretrained(checkpoint)
model_ft = AutoModelForCausalLM.from_pretrained(checkpoint, torch_dtype=torch.bfloat16, device_map="auto")

In [15]:
# Test the fine-tuned model on the same prompt

# Let's test the base model before training
prompt = "Write a python programming language"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer_ft.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer_ft(formatted_prompt, return_tensors="pt").to(device)

outputs = model_ft.generate(**inputs, max_new_tokens=100)
print("Before training:")
print(tokenizer_ft.decode(outputs[0],skip_special_tokens=True))

Before training:
user
Write a python programming language
Write a python programming language
Write a python programming language
Write a python programming language
Write a python programming language
Write a python programming language
Write a python programming language
Write a python programming language
Write a python programming language
Write a python programming language
Write a python programming language
Write a python programming language
Write a python programming language
Write a python programming language
Write a python programming language
Write a python programming language
Write a python programming language
Write a python programming
